In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch, ArrowStyle, Arc
from mpl_toolkits.mplot3d import proj3d
from mpl_toolkits.mplot3d import axes3d    
import numpy as np
from scipy.spatial.transform import Rotation as R

from thesis_utils.plotting import set_plotting, save_figure

from gwviz.mplutils import add_arrow3d
from pypdf import PdfWriter, PdfReader

set_plotting()

In [ ]:
def angle(a, b):
    return np.arccos(np.dot(a, b) / np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
s1 = np.array([0.15, 0.25, 0.2])
s2 = np.array([0.1, 0.2, 0.1])
L = np.array([0, 0, 1.0])
J = (s1 + s2) + L

bases = np.array([
    [1, 0, 0,],
    [0, 1, 0,],
    [0, 0, 1,],
])
n = bases[2]


mass_angle = 1.3 * np.pi

In [ ]:
iota = np.pi / 8
psi = 0.6 * np.pi

r = R.from_euler("yz", [iota, psi], degrees=False)

In [ ]:
orbit_radius = 1
theta = np.linspace(0, 2 * np.pi, 100)
orbit = np.array([
    orbit_radius * np.cos(theta),
    orbit_radius * np.sin(theta),
    np.zeros(len(theta)),
])

orbit_radii = np.linspace(0, orbit_radius, 100)
orbit_plane = [
    np.outer(orbit_radii, np.cos(theta)),
    np.outer(orbit_radii, np.sin(theta)),
]

In [ ]:
m1_coords = np.array([
    orbit_radius * np.cos(mass_angle),
    orbit_radius * np.sin(mass_angle),
    0.0
])
m2_coords = np.array([
    orbit_radius * np.cos(mass_angle + np.pi),
    orbit_radius * np.sin(mass_angle + np.pi),
    0.0
])

m1_coords_trans = r.apply(m1_coords)
m2_coords_trans = r.apply(m2_coords)

s1_coords_trans = r.apply(m1_coords + s1)
s2_coords_trans = r.apply(m2_coords + s2)

In [ ]:
L_angle_distance = 0.5
L_angle_end = L_angle_distance * L
L_angle_end_trans = r.apply(L_angle_end)

L_angle = angle(L_angle_end_trans, n)

J_angle_distance = 0.5
J_angle_end = J_angle_distance * J
J_angle_end_trans = r.apply(J_angle_end)

theta_jn = angle(J_angle_end_trans, n)

J_angle_y = angle(J_angle_end, bases[1])
J_rot = r.apply(J)
J_angle = np.arctan2(J_rot[1], J_rot[0])
print(J_angle)

r_J = R.from_euler("z", [J_angle], degrees=False)
r_L = R.from_euler("z", [psi], degrees=False)

theta = np.linspace(0, L_angle, 100)
L_orbit = np.array([
    L_angle_distance * np.sin(theta),
    np.zeros(len(theta)),
    L_angle_distance * np.cos(theta),
])


theta = np.linspace(0, theta_jn, 100)
J_orbit = np.array([
    J_angle_distance * np.sin(theta),
    np.zeros(len(theta)),
    J_angle_distance * np.cos(theta),
])

In [ ]:
rotated_orbit = r.apply(orbit.T).T

In [ ]:
class Arrow3D(FancyArrowPatch):
    def __init__(self, start, end, *args, **kwargs):
        xs = np.array([start[0], end[0]])
        ys = np.array([start[1], end[1]])
        zs = np.array([start[2], end[2]])
        print(xs, ys, zs)
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def do_3d_projection(self, renderer=None):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, self.axes.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))

        return np.min(zs)

In [ ]:
text_zorder = 5
orbit_zorder = -1

axis_labels = [r"$\hat{\mathbf{x}}$", r"$\hat{\mathbf{y}}$", r"$\hat{\mathbf{n}}$"]
prime_axis_labels = [r"$\hat{\mathbf{x}}'$", r"$\hat{\mathbf{y}}'$", None]
label_pad = np.array([0, 0, 0.08])
label_prime_pad = np.array([0, 0, -0.08])

J_pad = np.array([0, 0, 0.075])
L_pad = np.array([0, 0, 0.075])
s_pad = 0.08 * np.array([0, 0, 1])
m1_pad = 0.12 * np.array([-1, 1, 0])
m2_pad = 0.12 * np.array([1, -1, 0])

n_angle = 100
psi_pos = 1 / 3
psi_pad = np.array([0, 0, -0.08])
iota_pos = 1 / 3
iota_pad = np.array([0, 0, -0.08])
theta_jn_pos = 0.4
theta_jn_pad = np.array([0, 0, 0.09])

In [ ]:
fig = plt.figure(dpi=200)
ax = fig.add_subplot(111, projection="3d", computed_zorder=False)
ax.view_init(vertical_axis="z", azim=45, elev=None)
ax.set_axis_off()

axis_len = 1

ex_rot, ey_rot, ez_rot = r.apply(np.eye(3))

origin = np.zeros(3)

arrow_kwargs = dict(
    shrinkA=0.0,
    shrinkB=0.0,
    arrowstyle=ArrowStyle.Fancy(head_length=2.0, head_width=2.0),
    # mutation_scale=100.0,
)
line_kwargs = dict(
    shrinkA=0.0,
    shrinkB=0.0,
    lw=0.5,
    arrowstyle="-",
    zorder=1,
)

angle_kwargs = dict(
    lw=1.0,
)
# Orbits
ax.plot(*orbit, color="grey", zorder=orbit_zorder)
ax.plot(*rotated_orbit, zorder=orbit_zorder)

for b in bases:
    add_arrow3d(ax, origin, axis_len * b, lw=1.0, color="grey", **arrow_kwargs)

for b in [ex_rot, ey_rot]:
    add_arrow3d(ax, origin, axis_len * b, lw=1.0, color="lightblue", **arrow_kwargs)


# Components
ax.scatter(*m1_coords_trans, s=50.0, color="k")
ax.scatter(*m2_coords_trans, s=50.0, color="k")
ax.text(
    *m1_coords_trans + m1_pad,
    "$m_1$",
    verticalalignment="center",
    horizontalalignment="center",
)
ax.text(
    *m2_coords_trans + m2_pad,
    "$m_2$",
    verticalalignment="center",
    horizontalalignment="center",
)

# Spins
ax.add_artist(
    Arrow3D(m1_coords_trans, s1_coords_trans, color="k", **arrow_kwargs)
)
ax.add_artist(
    Arrow3D(m2_coords_trans, s2_coords_trans, color="k", **arrow_kwargs)
)
ax.text(
    *s1_coords_trans + s_pad,
    "$\mathbf{s_1}$",
    verticalalignment="center",
    horizontalalignment="center",
)
ax.text(
    *s2_coords_trans + s_pad,
    "$\mathbf{s_2}$",
    verticalalignment="center",
    horizontalalignment="center",
)

# Momentum vectors
L_rot = r.apply(L)
add_arrow3d(
    ax, origin, L_rot, color="orange", **arrow_kwargs
)
J_rot = r.apply(J)
add_arrow3d(
    ax, origin, J_rot, color="red", ls="-", **arrow_kwargs
)

ax.text(
    *J_rot + J_pad,
    "$\mathbf{J}$",
    verticalalignment="center",
    horizontalalignment="center",
)
ax.text(
    *L_rot + L_pad,
    "$\mathbf{L}$",
    verticalalignment="center",
    horizontalalignment="center",
)

# Angles
L_angle_line = r_L.apply(L_orbit.T)
J_angle_line = r_J.apply(J_orbit.T)

ax.plot(*L_angle_line.T, color="orange", **angle_kwargs)
ax.plot(*J_angle_line.T, color="red", **angle_kwargs)

ax.text(
    *L_angle_line[int(iota_pos * n_angle)] + iota_pad,
    r"$\iota$",
    verticalalignment="center",
    horizontalalignment="center",
    zorder=text_zorder
)
ax.text(
    *J_angle_line[int(theta_jn_pos * n_angle)] + theta_jn_pad,
    r"$\theta_\textrm{JN}$",
    verticalalignment="center",
    horizontalalignment="center",
    zorder=text_zorder
)

# Psi
psi_line_rot = ex_rot.copy()
psi_line_rot[2] = 0.0
psi_line_rot /= np.linalg.norm(psi_line_rot)

add_arrow3d(
    ax, origin, psi_line_rot, color="purple", ls=":", **line_kwargs
)
add_arrow3d(
    ax, psi_line_rot, ex_rot, color="purple", ls=":", **line_kwargs
)

psi_vec = np.linspace(0, psi, n_angle)
psi_r = 0.4
psi_angle = np.array([
    psi_r * np.cos(psi_vec),
    psi_r * np.sin(psi_vec),
    np.zeros(len(psi_vec))
])
ax.plot(*psi_angle, color="purple", **angle_kwargs)
ax.text(
    *psi_angle.T[int(psi_pos * n_angle)] + psi_pad,
    "$\psi$",
    verticalalignment="center",
    horizontalalignment="center",
)


for b, label in zip(bases, axis_labels):
    ax.text(
        *axis_len * b + label_pad,
        label,
        verticalalignment="center",
        horizontalalignment="center",
    )

for b, label in zip([ex_rot, ey_rot], prime_axis_labels):
    ax.text(
        *axis_len * b + label_prime_pad,
        label,
        verticalalignment="center",
        horizontalalignment="center",
        zorder=text_zorder,
    )

xlims = [-1, 1]
ylims = [-1, 1]
zlims = [0, 1.0]

ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_zlim(zlims)

ax.set_box_aspect((np.ptp(xlims), np.ptp(ylims), np.ptp(zlims)), zoom=1.0)

plt.show()
save_figure(fig, "cbc_diagram", "figures")

In [ ]:
with open("figures/cbc_diagram.pdf","rb") as f:
    orig = PdfReader(f)
    cropped = PdfWriter()

    image = orig.pages[0]
    top = 55
    bottom = 28
    left = 30
    right = 30

    image.mediabox.upper_right = (
        image.mediabox.right - right, image.mediabox.top - top,
    )
    image.mediabox.lower_left = (
        image.mediabox.left + left, image.mediabox.bottom + bottom,
    )
    cropped.add_page(image)


with open("figures/cbc_diagram_cropped.pdf", "wb") as f:
    cropped.write(f)